In [1]:
import numpy as np
import os
dtype = np.float32

import torch
import torchvision
from tqdm import tqdm
import os

from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

import glob
import random
import json
import os

def contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)

/home/jgryu/miniconda3/envs/RD/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Weight Param 이미지 size로 만들기
JPEG, image pretrained nic model에 사용

path 파일로 split해서 사용하자 그냥

In [3]:
# json_path1 = '/home/jgryu/Weight_compression/Wparam_dataset/path_json/meta-llama-3-8b_mlp_train.json'
# json_path2 = '/home/jgryu/Weight_compression/Wparam_dataset/path_json/meta-llama-3-8b_attn_train.json'
json_path1 = '/home/jgryu/Weight_compression/Wparam_dataset/path_json/llama-2-7b_mlp_train.json'
json_path2 = '/home/jgryu/Weight_compression/Wparam_dataset/path_json/llama-2-7b_attn_train.json'
save_dir_path = '/home/jgryu/Weight_compression/Wparam_dataset/image_shape_wp/'

for json_path in [json_path1, json_path2]:
    print(f'### {json_path} ###')
    for d in [64, 256, 512, 1024]:
    # for d in [256]:
        print(f'## {d} ##')
        dim = (d, d)
        file_name = os.path.splitext(os.path.basename(json_path))[0]
        file_name = file_name  + "_json"

        save_path = os.path.join(save_dir_path, file_name, f'{dim[0]}_{dim[1]}')
        os.makedirs(save_path, exist_ok=True)

        with open(json_path, 'r') as f:
            w_path = json.load(f)

        print(len(w_path))

        w_list = []
        for wp in tqdm(w_path):
            w  = np.load(wp)
            w = w.reshape(-1, *dim)
            
            # "/home/jgryu/Weight_compression/Wparam_dataset/model_param_tensor/meta-llama/Meta-Llama-3-8B/model-layers-3-mlp-down_proj-weight.npy"
            # 디렉토리의 마지막 부분과 파일명 추출
            dir_name = os.path.basename(os.path.dirname(wp))  # "Meta-Llama-3-8B"
            file_name = os.path.splitext(os.path.basename(wp))[0]  # "model-layers-3-mlp-down_proj-weight"    
            path = os.path.join(save_path, f"{dir_name}/{file_name}_npy")
            os.makedirs(path, exist_ok=True)
            for i, wi in enumerate(w):
                np.save(f'{path}/{i}', wi)
            w_list.append(w)
            

        w_list = np.concatenate(w_list, axis = 0)
        print(w_list.shape)


        mean_vector = w_list.mean(axis=0)
        mean_value = w_list.mean()
        std_vector = w_list.std(axis=0)
        std_value = w_list.std()
        print(mean_vector.shape, std_vector.shape, mean_value, std_value)

        filename = save_path
        np.save(filename + '/mean_vector.npy', mean_vector)
        np.save(filename + '/mean_value.npy', mean_value)
        np.save(filename + '/std_vector.npy', std_vector)
        np.save(filename + '/std_value.npy', std_value)


### /home/jgryu/Weight_compression/Wparam_dataset/path_json/llama-2-7b_mlp_train.json ###
## 64 ##
76


100%|██████████| 76/76 [00:54<00:00,  1.39it/s]


(836608, 64, 64)
(64, 64) (64, 64) -4.6755717e-06 0.018473955
## 256 ##
76


100%|██████████| 76/76 [00:13<00:00,  5.59it/s]


(52288, 256, 256)
(256, 256) (256, 256) -4.6755717e-06 0.018473955
## 512 ##
76


...

(26112, 256, 256)
(256, 256) (256, 256) 3.2762267e-07 0.02040687
## 512 ##
102


100%|██████████| 102/102 [00:04<00:00, 23.01it/s]


(6528, 512, 512)
(512, 512) (512, 512) 3.2762267e-07 0.02040687
## 1024 ##
102


100%|██████████| 102/102 [00:04<00:00, 20.89it/s]


(1632, 1024, 1024)
(1024, 1024) (1024, 1024) 3.2762267e-07 0.02040687
